In [ ]:
def build_model(input_shape=(224, 224, 3)):
    inputs = Input(input_shape)

    #Contracting Path
    layer_ = []
    buf_in = inputs
    for size_ in [64, 128, 256, 512, 1024]:
        conv = Conv2D(size_, 3, activation='relu', padding='same',kernel_initializer='he_normal')(buf_in)
        conv = Conv2D(size_, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
        if size_ >= 512:
            conv = Dropout(0.5)(conv)
        if size_ == 1024:
            buf_in = conv
            layer_.append(conv)
            break
        layer_.append(conv)
        pool = MaxPooling2D(pool_size=(2, 2))(conv)
        buf_in = pool
    layer_.reverse()

    # Contracting Path 풀어 놓은것
    # conv1 = Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='he_normal')(inputs)
    # conv1 = Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv1)
    # pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # conv2 = Conv2D(128, 3, activation='relu', padding='same',kernel_initializer='he_normal')(pool1)
    # conv2 = Conv2D(128, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv2)
    # pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # conv3 = Conv2D(256, 3, activation='relu', padding='same',kernel_initializer='he_normal')(pool2)
    # conv3 = Conv2D(256, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv3)
    # pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # conv4 = Conv2D(512, 3, activation='relu', padding='same',kernel_initializer='he_normal')(pool3)
    # conv4 = Conv2D(512, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv4)
    # drop4 = Dropout(0.5)(conv4)
    # pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # conv5 = Conv2D(1024, 3, activation='relu', padding='same',kernel_initializer='he_normal')(pool4)  
    # conv5 = Conv2D(1024, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv5)
    # drop5 = Dropout(0.5)(conv5)
    
    #Expanding Path
    buf_in = layer_[0]
    for idx, size_ in enumerate([512, 256, 128, 64]):
        up_ = Conv2DTranspose(size_, 2, activation='relu', strides=(2,2), kernel_initializer='he_normal')(buf_in)
        merge_ = concatenate([layer_[idx+1], up_], axis = 3)
        conv = Conv2D(size_, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge_)
        conv = Conv2D(size_, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
        buf_in = conv

    conv9 = Conv2D(2, 3, activation='relu', padding='same',kernel_initializer='he_normal')(buf_in)     
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs = inputs, outputs = conv10)
    return model

    # Expanding Path 풀어 놓은것
    # up6 = Conv2DTranspose(512, 2, activation='relu', strides=(2,2), kernel_initializer='he_normal')(drop5)
    # merge6 = concatenate([drop4,up6], axis = 3)
    # conv6 = Conv2D(512, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge6)
    # conv6 = Conv2D(512, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv6)

    # up7 = Conv2DTranspose(256, 2, activation='relu', strides=(2,2), kernel_initializer='he_normal')(conv6)
    # merge7 = concatenate([conv3,up7], axis = 3)
    # conv7 = Conv2D(256, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge7)
    # conv7 = Conv2D(256, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv7)

    # up8 = Conv2DTranspose(128, 2, activation='relu', strides=(2,2), kernel_initializer='he_normal')(conv7)
    # merge8 = concatenate([conv2,up8], axis = 3)
    # conv8 = Conv2D(128, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge8)
    # conv8 = Conv2D(128, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv8)

    # up9 = Conv2DTranspose(64, 2, activation='relu', strides=(2,2), kernel_initializer='he_normal')(conv8)
    # merge9 = concatenate([conv1,up9], axis = 3)
    # conv9 = Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge9)
    # conv9 = Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv9)  

    # conv9 = Conv2D(2, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv9)     
    # conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    # model = Model(inputs = inputs, outputs = conv10)
    # return model


In [ ]:
def build_model_adv(input_shape=(224, 224, 3)):
    inputs = Input(input_shape)

    #Contracting Path
    filter = [64, 128, 256, 512, 1024]
    layer_backbone = []
    buf_in = inputs
    for size_ in filter:
        conv = Conv2D(size_, 3, activation='relu', padding='same',kernel_initializer='he_normal')(buf_in)
        conv = Conv2D(size_, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
        # if size_ >= 512:
        #     conv = Dropout(0.5)(conv)
        if size_ == 1024:
            layer_backbone.append(conv)
            buf_in = conv
            break
        layer_backbone.append(conv)
        pool = MaxPooling2D(pool_size=(2, 2))(conv)
        buf_in = pool
    layer_backbone.reverse()

    
    filter = filter[:-1]
    filter.reverse()
    layer_up = []
    # Skip Connection
    num_loop = len(layer_backbone[1:])
    for idx in range(1, num_loop):
        num_filter = filter[idx]
        up_ = Conv2DTranspose(num_filter, (2, 2), strides=(2, 2), name=f'x_{num_loop-idx}_{idx}', padding='same')(layer_backbone[idx])
        merge_ = concatenate([up_, layer_backbone[idx+1]], name=f'merge_{num_loop-idx}_{idx}', axis=3)
        conv = Conv2D(num_filter, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge_)
        conv = Conv2D(num_filter, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
        layer_up.append(conv)

        for cnt in range(idx-1):
            up_ = Conv2DTranspose(num_filter, (2, 2), strides=(2, 2), name=f'x_{num_loop-idx}_{idx+cnt+1}', padding='same')(layer_up[idx+cnt-2])
            list_merge = []
            list_merge.append(up_)
            list_merge.append(layer_backbone[idx+1])
            for i in range(cnt+1):
                list_merge.append(layer_up[2*idx-3+i])
            merge_ = concatenate(list_merge, name=f'merge_{num_loop-idx}_{idx+cnt+1}', axis=3)
            conv = Conv2D(num_filter, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge_)
            conv = Conv2D(num_filter, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
            layer_up.append(conv)


    # Skip Connection 풀어 놓은것
    # # layer_up[0] 
    # up_ = Conv2DTranspose(256, (2, 2), strides=(2, 2), name='up32', padding='same')(layer_backbone[1])
    # merge_ = concatenate([up_, layer_backbone[2]], name='merge32', axis=3)
    # conv = Conv2D(256, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge_)
    # conv = Conv2D(256, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
    # layer_up.append(conv)

    # # layer_up[1]
    # up_ = Conv2DTranspose(128, (2, 2), strides=(2, 2), name='up32', padding='same')(layer_backbone[2])
    # merge_ = concatenate([up_, layer_backbone[3]], name='merge32', axis=3)
    # conv = Conv2D(128, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge_)
    # conv = Conv2D(128, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
    # layer_up.append(conv)

    # # layer_up[2]
    # up_ = Conv2DTranspose(128, (2, 2), strides=(2, 2), name='up32', padding='same')(layer_up[0])
    # merge_ = concatenate([up_, layer_backbone[3], layer_up[1]], name='merge32', axis=3)
    # conv = Conv2D(128, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge_)
    # conv = Conv2D(128, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
    # layer_up.append(conv)

    # # layer_up[3]
    # up_ = Conv2DTranspose(64, (2, 2), strides=(2, 2), name='up32', padding='same')(layer_backbone[3])
    # merge_ = concatenate([up_, layer_backbone[4]], name='merge32', axis=3)
    # conv = Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge_)
    # conv = Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
    # layer_up.append(conv)

    # # layer_up[4]
    # up_ = Conv2DTranspose(64, (2, 2), strides=(2, 2), name='up32', padding='same')(layer_up[1])
    # merge_ = concatenate([up_, layer_backbone[4], layer_up[3]], name='merge32', axis=3)
    # conv = Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge_)
    # conv = Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
    # layer_up.append(conv)

    # # layer_up[5]
    # up_ = Conv2DTranspose(64, (2, 2), strides=(2, 2), name='up32', padding='same')(layer_up[2])
    # merge_ = concatenate([up_, layer_backbone[4], layer_up[3], layer_up[4]], name='merge32', axis=3)
    # conv = Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='he_normal')(merge_)
    # conv = Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='he_normal')(conv)
    # layer_up.append(conv)


    # Upsampling
    # Expanding Path
    idx_up = 0
    buf_in = layer_backbone[0]
    for idx, size_ in enumerate(filter):
        up_ = Conv2DTranspose(size_, 2, name=f'up_{idx}', activation='relu', strides=(2,2), kernel_initializer='he_normal')(buf_in)
        list_merge = []
        list_merge.append(up_)
        list_merge.append(layer_backbone[idx+1])
        for _ in range(idx):
            list_merge.append(layer_up[idx_up])
            idx_up+=1
        print(idx, 'aaa', list_merge)
        merge_ = concatenate(list_merge, name=f'merge_{idx}', axis = 3)
        conv = Conv2D(size_, 3, name=f'conv_{idx}_1', activation='relu', padding='same',kernel_initializer='he_normal')(merge_)
        conv = Conv2D(size_, 3, name=f'conv_{idx}_2', activation='relu', padding='same',kernel_initializer='he_normal')(conv)
        buf_in = conv

    # output_1 = Conv2D(1, 1, activation='sigmoid', name='output_1', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_2)
    
    output_1 = Conv2D(1, 1, activation='sigmoid', name='output_1', kernel_initializer = 'he_normal', padding='same')(layer_up[3])
    output_2 = Conv2D(1, 1, activation='sigmoid', name='output_2', kernel_initializer = 'he_normal', padding='same')(layer_up[4])
    output_3 = Conv2D(1, 1, activation='sigmoid', name='output_3', kernel_initializer = 'he_normal', padding='same')(layer_up[5])
    output_4 = Conv2D(1, 1, activation='sigmoid', name='output_4', kernel_initializer = 'he_normal', padding='same')(buf_in)
    model = Model(inputs = inputs, outputs = [output_4])
    # model = Model(inputs = inputs, outputs = [output_1, output_2, output_3, output_4])
    return model


In [ ]:
model_unet = build_model()
model_unet.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 224, 224, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
model_unet_adv = build_model_adv()
model_unet_adv.summary()


0 aaa [<KerasTensor: shape=(None, 28, 28, 512) dtype=float32 (created by layer 'up_0')>, <KerasTensor: shape=(None, 28, 28, 512) dtype=float32 (created by layer 'conv2d_27')>]
1 aaa [<KerasTensor: shape=(None, 56, 56, 256) dtype=float32 (created by layer 'up_1')>, <KerasTensor: shape=(None, 56, 56, 256) dtype=float32 (created by layer 'conv2d_25')>, <KerasTensor: shape=(None, 56, 56, 256) dtype=float32 (created by layer 'conv2d_31')>]
2 aaa [<KerasTensor: shape=(None, 112, 112, 128) dtype=float32 (created by layer 'up_2')>, <KerasTensor: shape=(None, 112, 112, 128) dtype=float32 (created by layer 'conv2d_23')>, <KerasTensor: shape=(None, 112, 112, 128) dtype=float32 (created by layer 'conv2d_33')>, <KerasTensor: shape=(None, 112, 112, 128) dtype=float32 (created by layer 'conv2d_35')>]
3 aaa [<KerasTensor: shape=(None, 224, 224, 64) dtype=float32 (created by layer 'up_3')>, <KerasTensor: shape=(None, 224, 224, 64) dtype=float32 (created by layer 'conv2d_21')>, <KerasTensor: shape=(None